### INSURANCE

Seguiremos el mismo pasa para esta variable pero analizaremos que es lo que opina la gente sobre los seguros de salud en USA

In [30]:
import praw
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [31]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juand\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\juand\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\juand\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [32]:
reddit = praw.Reddit(
    client_id='vkbyNxlrdz1Wy9DNxfxnzA',
    client_secret='sC4RfUNF95ThEF0BtuTyfD1sV6DyWA',
    user_agent='my-app by u/Witty-Cause-3665'
)

subreddit = reddit.subreddit('health')
posts = subreddit.search("insurance USA", limit=20
                        )
comments_data = []

for post in posts:
    post.comments.replace_more(limit=None)
    for comment in post.comments.list():
        comments_data.append(comment.body)

In [33]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

sia = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    sentiment_score = sia.polarity_scores(text)
    return sentiment_score['compound']



In [34]:
sia = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    sentiment_score = sia.polarity_scores(text)
    return sentiment_score['compound']
comments_processed = [preprocess_text(comment) for comment in comments_data]
sentiments = [analyze_sentiment(comment) for comment in comments_processed]
sentiment_labels = ['Positive' if score > 0.05 else 'Negative' if score < -0.05 else 'Neutral' for score in sentiments]

data = pd.DataFrame({'Comment Body': comments_data, 'Sentiment': sentiment_labels})

X_train, X_test, y_train, y_test = train_test_split(data['Comment Body'], data['Sentiment'], test_size=0.2, random_state=42)

tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

svm_model = SVC(kernel='linear')
svm_model.fit(X_train_tfidf, y_train)

y_pred = svm_model.predict(X_test_tfidf)

results_df_Insurance = pd.DataFrame({'Comment Body': X_test, 'Actual Sentiment': y_test, 'Predicted Sentiment': y_pred})

results_df_Insurance

,Comment Body,Actual Sentiment,Predicted Sentiment
231,The idiom.,Neutral,Positive
110,Pretty high yes. These articles seem to forget...,Positive,Positive
250,"I'm sorry, which country has the higher surviv...",Negative,Negative
9,"Hey folks, the cranberry juice thing? You want...",Positive,Positive
93,Not that this is realistic... but would you pa...,Negative,Positive
...,...,...,...
248,This writer doesn't know the difference betwee...,Positive,Positive
60,"The MD *pays* them himself, or *bills her* for...",Positive,Positive
124,Ha!,Positive,Neutral
261,[Pancreatic cancer mortality rates are about t...,Negative,Negative


In [35]:
results_df_Insurance.to_csv('results_df_Insurance.csv', index=False)